<a href="https://colab.research.google.com/github/silverkkang/DL/blob/main/bertopic251118.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install pandas scikit-learn
!pip install transformers torch
!pip install scikit-learn matplotlib
!pip install bertopic

In [ ]:
import pandas as pd
import numpy as np
import re
import pickle
import os
from datetime import datetime
from sklearn.model_selection import train_test_split

In [ ]:
file_name = '/content/drive/MyDrive/Colab Notebooks/진짜 최종종합본.csv'
df = pd.read_csv(file_name, engine='python', on_bad_lines='skip')

In [ ]:
print(df.head(3))

   Unnamed: 0                                 title  \
0           0          소음순 돌출 되어 불편함 겪어보신 분 계실까요..?   
1           1                        임부복 바지 도와주세요 ㅠ   
2           2  모유수유 분유 상관 없이 수유쿠션은 쓰나요? 둘중 뭐가 편할까요!   

                                            contents  \
0  지금 임신 5주차예요 ~예전에 산부인과에서 검사하면서소음순이 비대칭이네 라고 원장님...   
1  스토퍼형 하나사봤는데 조이면 조인곳이 너무 불편하고안조이면 흘러내리고 ㅜ원피스 사긴...   
2         가격 상관 없이 뭐가 나아보이나요?수유 어떻게 할지 모르겠는데 잘쓸까요,,?   

                                                 url        date  nouns  
0  https://cafe.naver.com/f-e/cafes/10094499/arti...  2025-11-15    NaN  
1  https://cafe.naver.com/f-e/cafes/10094499/arti...  2025-11-15    NaN  
2  https://cafe.naver.com/f-e/cafes/10094499/arti...  2025-11-15    NaN  


# Step1. 전처리


In [ ]:
TEXT_COLUMN = 'contents'
original_count = len(df)

In [ ]:
#1 결측치 제거
df = df.dropna(subset=[TEXT_COLUMN])
print(f"1️⃣  결측치 제거: {original_count - len(df):,}개 → {len(df):,}개 남음")

# 빈 문자열 제거
before_count = len(df)
df = df[df[TEXT_COLUMN].str.strip() != '']
print(f"2️⃣  빈 문자열 제거: {before_count - len(df):,}개 → {len(df):,}개 남음")

# 정규표현식 정리
def clean_text(text):
    if not isinstance(text, str):
        return ""
    text = re.sub(r'<[^>]+>', '', text)  # HTML
    text = re.sub(r'http[s]?://\S+', '', text)  # URL
    text = re.sub(r'\S+@\S+', '', text)  # Email
    text = re.sub(r'([!?])\1{2,}', r'\1', text)  # 특수문자
    text = re.sub(r'(\.)\1{3,}', r'\1\1\1', text)
    text = re.sub(r'([ㅋㅎㅠㅜㅡ])\1{2,}', r'\1\1', text)  # 한글 자음
    text = re.sub(r'\s+', ' ', text).strip()
    return text

df[TEXT_COLUMN] = df[TEXT_COLUMN].apply(clean_text)
df = df[df[TEXT_COLUMN] != '']

# 중복 제거
before_count = len(df)
df = df.drop_duplicates(subset=[TEXT_COLUMN], keep='first')

# 최소 길이 필터링
MIN_LENGTH = 5
before_count = len(df)
df = df[df[TEXT_COLUMN].str.len() >= MIN_LENGTH]

df = df.reset_index(drop=True)

1️⃣  결측치 제거: 33개 → 90,160개 남음
2️⃣  빈 문자열 제거: 1개 → 90,159개 남음


In [ ]:

print(f"\n전처리 완료: {original_count:,}개 → {len(df):,}개")



전처리 완료: 90,193개 → 90,104개


In [ ]:
# date 컬럼을 datetime으로 변환
df['date'] = pd.to_datetime(df['date'])

In [ ]:
# 연도-분기 컬럼 생성 (예: '2022-Q3', '2025-Q4')
df['year_quarter'] = df['date'].dt.year.astype(str) + '-Q' + df['date'].dt.quarter.astype(str)

In [ ]:
# 분기별 분포 확인
print("\n분기별 데이터 분포:")
print("-" * 50)
quarter_counts = df['year_quarter'].value_counts().sort_index()
for quarter, count in quarter_counts.items():
    percentage = count / len(df) * 100
    bar = "█" * int(percentage / 2)
    print(f"{quarter}  |  {count:6,}개  ({percentage:5.1f}%)  {bar}")
print("-" * 50)
print(f"총 {len(quarter_counts)}개 분기")


분기별 데이터 분포:
--------------------------------------------------
2021.0-Q1.0  |      11개  (  0.0%)  
2021.0-Q2.0  |     189개  (  0.2%)  
2021.0-Q3.0  |     286개  (  0.3%)  
2021.0-Q4.0  |     353개  (  0.4%)  
2022.0-Q1.0  |     475개  (  0.5%)  
2022.0-Q2.0  |     621개  (  0.7%)  
2022.0-Q3.0  |     807개  (  0.9%)  
2022.0-Q4.0  |     896개  (  1.0%)  
2023.0-Q1.0  |     973개  (  1.1%)  
2023.0-Q2.0  |     895개  (  1.0%)  
2023.0-Q3.0  |   1,093개  (  1.2%)  
2023.0-Q4.0  |     887개  (  1.0%)  
2024.0-Q1.0  |   1,024개  (  1.1%)  
2024.0-Q2.0  |   1,373개  (  1.5%)  
2024.0-Q3.0  |   2,098개  (  2.3%)  █
2024.0-Q4.0  |   1,915개  (  2.1%)  █
2025.0-Q1.0  |   2,374개  (  2.6%)  █
2025.0-Q2.0  |   4,291개  (  4.8%)  ██
2025.0-Q3.0  |   8,075개  (  9.0%)  ████
2025.0-Q4.0  |   3,916개  (  4.3%)  ██
nan-Qnan  |  57,552개  ( 63.9%)  ███████████████████████████████
--------------------------------------------------
총 21개 분기


In [ ]:
#1층화 기준 설정
STRATIFY_COLUMN = None
print(f"사용 가능한 컬럼: {df.columns.tolist()}")

사용 가능한 컬럼: ['Unnamed: 0', 'title', 'contents', 'url', 'date', 'nouns']


In [ ]:
SAMPLE_SIZE = 10000

if len(df) > SAMPLE_SIZE:
    print(f"전처리 후 데이터: {len(df):,}개")
    print(f"샘플링 목표: {SAMPLE_SIZE:,}개")

    # 샘플링 비율 계산
    sample_ratio = SAMPLE_SIZE / len(df)

    try:
        # 분기별 층화 샘플링
        df_sampled, _ = train_test_split(
            df,
            train_size=sample_ratio,
            stratify=df['year_quarter'],
            random_state=42
        )
        df_sampled = df_sampled.sort_index().reset_index(drop=True)

        print(f"\n✅ 층화 샘플링 완료!")
        print(f"   - 전처리 후: {len(df):,}개")
        print(f"   - 샘플 추출: {len(df_sampled):,}개")
        print(f"   - 샘플링 비율: {sample_ratio*100:.1f}%")

        # 샘플 후 분기별 분포 확인
        print("\n샘플 후 분기별 분포:")

        sampled_quarter_counts = df_sampled['year_quarter'].value_counts().sort_index()
        for quarter in quarter_counts.index:
            original = quarter_counts[quarter]
            sampled = sampled_quarter_counts.get(quarter, 0)
            ratio = sampled / original * 100 if original > 0 else 0
            print(f"{quarter}  |  {original:6,}개 → {sampled:5,}개  ({ratio:5.1f}%)")


        # 비율이 유지되는지 검증
        print("\n층화 추출 검증:")
        print(f"   전체 샘플링 비율: {sample_ratio*100:.2f}%")
        print(f"   각 분기 샘플링 비율: 약 {sample_ratio*100:.2f}% (균일)")

    except ValueError as e:
        print(f"\n⚠️  층화 추출 실패: {e}")
        print("   → 랜덤 샘플링으로 대체")
        df_sampled = df.sample(n=SAMPLE_SIZE, random_state=42).reset_index(drop=True)

else:
    print(f"⚠️  데이터가 {SAMPLE_SIZE:,}개 이하입니다.")
    print(f"   전체 {len(df):,}개 사용")
    df_sampled = df.copy()


In [ ]:
print("\n📝 샘플링 후 데이터 확인 (5개):")
for i, row in df_sampled.head(5).iterrows():
    text = row[TEXT_COLUMN]
    quarter = row['year_quarter']
    print(f"{i+1}. [{quarter}] {text[:80]}{'...' if len(text) > 80 else ''}")

texts = df_sampled[TEXT_COLUMN].tolist()
print(f"\n최종 학습 데이터: {len(texts):,}개")

In [ ]:
save_data_step1 = {
    'df_original': df,  # 전처리 후 전체
    'df_sampled': df_sampled,  # 샘플링된 데이터
    'TEXT_COLUMN': TEXT_COLUMN,
    'stratify_column': 'year_quarter',
    'quarter_distribution_original': quarter_counts.to_dict(),
    'quarter_distribution_sampled': df_sampled['year_quarter'].value_counts().to_dict(),
    'original_count': original_count,
    'preprocessed_count': len(df),
    'sample_count': len(df_sampled),
    'sampling_ratio': SAMPLE_SIZE / len(df) if len(df) > SAMPLE_SIZE else 1.0,
    'timestamp': datetime.now().strftime('%Y-%m-%d %H:%M:%S')
}

with open('step1_preprocessing_result.pkl', 'wb') as f:
    pickle.dump(save_data_step1, f)

file_size = os.path.getsize('step1_preprocessing_result.pkl') / (1024**2)
print(f"저장 완료: step1_preprocessing_result.pkl ({file_size:.1f} MB)")
print(f"""
저장 내용:
  - 전처리 전체 데이터: {len(df):,}개
  - 샘플링 데이터: {len(df_sampled):,}개
  - 텍스트 컬럼: {TEXT_COLUMN}
  - 층화 기준: 분기(year_quarter)
  - 분기 수: {len(quarter_counts)}개
""")

# Step 2. 임베딩

In [ ]:
# Step 2: 임베딩

import torch
from transformers import ElectraModel, ElectraTokenizer
from tqdm import tqdm

In [ ]:
#  2결과 불러오기
with open('step1_preprocessing_result.pkl', 'rb') as f:
    step1_data = pickle.load(f)

df_sampled = step1_data['df_sampled']
TEXT_COLUMN = step1_data['TEXT_COLUMN']
texts = df_sampled[TEXT_COLUMN].tolist()

In [ ]:
#모델 로드
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Device: {device}")

model_name = "monologg/koelectra-base-v3-discriminator"
tokenizer = ElectraTokenizer.from_pretrained(model_name)
model = ElectraModel.from_pretrained(model_name).to(device)
model.eval()

In [ ]:
#2텍스트 길이 분석
sample_size = min(1000, len(texts))
text_lengths = [len(tokenizer.encode(text)) for text in texts[:sample_size]]
avg_length = np.mean(text_lengths)
p95_length = np.percentile(text_lengths, 95)

print(f"   평균: {avg_length:.0f} 토큰")
print(f"   95%: {p95_length:.0f} 토큰")

max_length = min(int(p95_length), 512)
print(f"   → max_length 설정: {max_length}")

In [ ]:
#2임베딩 함수
def get_electra_embeddings(texts, batch_size=16, max_length=128):
    all_embeddings = []

    for i in tqdm(range(0, len(texts), batch_size), desc="Embedding"):
        batch_texts = texts[i:i+batch_size]

        inputs = tokenizer(
            batch_texts,
            padding=True,
            truncation=True,
            max_length=max_length,
            return_tensors="pt"
        ).to(device)

        with torch.no_grad():
            outputs = model(**inputs)

        # Mean Pooling
        last_hidden_states = outputs.last_hidden_state
        attention_mask = inputs['attention_mask']

        mask_expanded = attention_mask.unsqueeze(-1).expand(last_hidden_states.size()).float()
        sum_embeddings = torch.sum(last_hidden_states * mask_expanded, 1)
        sum_mask = torch.clamp(mask_expanded.sum(1), min=1e-9)
        mean_pooled_embeddings = sum_embeddings / sum_mask

        all_embeddings.extend(mean_pooled_embeddings.cpu().numpy())

        del inputs, outputs, last_hidden_states
        if device.type == 'cuda':
            torch.cuda.empty_cache()

    return np.array(all_embeddings)


In [ ]:
#2임베딩 생성
embeddings = get_electra_embeddings(texts, batch_size=16, max_length=max_length)

In [ ]:
#2피클 저장
save_data_step2 = {
    'embeddings': embeddings,
    'df_sampled': df_sampled,
    'texts': texts,
    'TEXT_COLUMN': TEXT_COLUMN,
    'max_length': max_length,
    'timestamp': datetime.now().strftime('%Y-%m-%d %H:%M:%S')
}

with open('step2_embeddings_result.pkl', 'wb') as f:
    pickle.dump(save_data_step2, f)

file_size = os.path.getsize('step2_embeddings_result.pkl') / (1024**2)
print(f"저장 완료: step2_embeddings_result.pkl ({file_size:.1f} MB)")
print(f"""
저장 내용:
  - 임베딩: {embeddings.shape}
  - 데이터프레임: {len(df_sampled):,}개
  - 텍스트: {len(texts):,}개
""")

# Step 3. 클러스터링

In [ ]:
#클러스터링

from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score
import matplotlib.pyplot as plt

In [ ]:
#결과 불러오기

with open('step2_embeddings_result.pkl', 'rb') as f:
    step2_data = pickle.load(f)

embeddings = step2_data['embeddings']
df_sampled = step2_data['df_sampled']
texts = step2_data['texts']
TEXT_COLUMN = step2_data['TEXT_COLUMN']

In [ ]:
#최적 K 찾기
max_k = min(15, len(texts) // 100)
k_range = range(2, max_k + 1)
silhouette_scores = []

for k in tqdm(k_range, desc="Finding K"):
    kmeans = KMeans(n_clusters=k, random_state=42, n_init=10, max_iter=300)
    clusters = kmeans.fit_predict(embeddings)
    score = silhouette_score(embeddings, clusters)
    silhouette_scores.append(score)
    print(f"K={k}: Silhouette Score = {score:.4f}")

In [ ]:
#시각화
plt.figure(figsize=(12, 7))
plt.plot(k_range, silhouette_scores, 'bo-', linewidth=2, markersize=10)
plt.xlabel('Number of Clusters (K)', fontsize=14)
plt.ylabel('Silhouette Score', fontsize=14)
plt.title('Optimal K Selection', fontsize=16, fontweight='bold')
plt.grid(True, alpha=0.3)
plt.xticks(k_range)

optimal_idx = np.argmax(silhouette_scores)
optimal_k = list(k_range)[optimal_idx]
optimal_score = silhouette_scores[optimal_idx]

plt.axvline(x=optimal_k, color='r', linestyle='--', linewidth=2, label=f'Optimal K={optimal_k}')
plt.scatter([optimal_k], [optimal_score], color='red', s=200, zorder=5)
plt.legend(fontsize=12)
plt.tight_layout()
plt.savefig('optimal_k_plot.png', dpi=300, bbox_inches='tight')
plt.show()

print(f"\n최적 K = {optimal_k} (Score: {optimal_score:.4f})")

In [ ]:
#최종 클러스터링
kmeans_final = KMeans(n_clusters=optimal_k, random_state=42, n_init=20, max_iter=300)
cluster_labels = kmeans_final.fit_predict(embeddings)

df_sampled['cluster'] = cluster_labels

In [ ]:
#클러스터 통계
cluster_counts = df_sampled['cluster'].value_counts().sort_index()
for i in range(optimal_k):
    count = cluster_counts[i]
    percentage = count / len(df_sampled) * 100
    bar = "█" * int(percentage / 2)
    print(f"Cluster {i}  |  {count:5,}개  ({percentage:5.1f}%)  {bar}")
print("-" * 50)

In [ ]:
#클러스터 샘플
print("\n📝 클러스터별 샘플 (각 5개):")
for i in range(optimal_k):
    print(f"\n[Cluster {i}]")
    samples = df_sampled[df_sampled['cluster'] == i][TEXT_COLUMN].head(5).tolist()
    for idx, sample in enumerate(samples, 1):
        print(f"  {idx}. {sample[:100]}...")

In [ ]:
#피클 저장
save_data_step3 = {
    'embeddings': embeddings,
    'df_clustered': df_sampled,
    'texts': texts,
    'TEXT_COLUMN': TEXT_COLUMN,
    'kmeans_model': kmeans_final,
    'cluster_labels': cluster_labels,
    'optimal_k': optimal_k,
    'optimal_score': optimal_score,
    'silhouette_scores': silhouette_scores,
    'k_range': list(k_range),
    'timestamp': datetime.now().strftime('%Y-%m-%d %H:%M:%S')
}

with open('step3_clustering_result.pkl', 'wb') as f:
    pickle.dump(save_data_step3, f)

In [ ]:
#CSV 저장
df_sampled.to_csv('clustering_result.csv', index=False, encoding='utf-8-sig')

file_size = os.path.getsize('step3_clustering_result.pkl') / (1024**2)
print(f"✅ Step 3 저장 완료:")
print(f"   - step3_clustering_result.pkl ({file_size:.1f} MB)")
print(f"   - clustering_result.csv")
print(f"""
저장 내용:
  - 클러스터링 데이터: {len(df_sampled):,}개
  - 클러스터 수: {optimal_k}개
  - Silhouette Score: {optimal_score:.4f}
""")


# Step4. BERTopic 분석

In [ ]:
from bertopic import BERTopic

In [ ]:
#결과 불러오기
with open('step3_clustering_result.pkl', 'rb') as f:
    step3_data = pickle.load(f)

df_clustered = step3_data['df_clustered']
TEXT_COLUMN = step3_data['TEXT_COLUMN']
optimal_k = step3_data['optimal_k']

print(f"불러오기 완료")
print(f"   데이터: {len(df_clustered):,}개")
print(f"   클러스터: {optimal_k}개")

In [ ]:
#클러스터별 BERTopic 수행
bertopic_results = {}

for cluster_id in range(optimal_k):

    # 클러스터 데이터 추출
    cluster_df = df_clustered[df_clustered['cluster'] == cluster_id]
    cluster_texts = cluster_df[TEXT_COLUMN].tolist()

    print(f"문서 수: {len(cluster_texts):,}개")

    if len(cluster_texts) < 10:
        print(f"문서가 너무 적어 BERTopic을 건너뜁니다.")
        continue

    try:
        # BERTopic 모델 생성
        topic_model = BERTopic(
            language='multilingual',
            calculate_probabilities=True,
            verbose=True,
            min_topic_size=min(10, len(cluster_texts) // 10)  # 동적 조정
        )

        # 토픽 추출
        topics, probs = topic_model.fit_transform(cluster_texts)

        # 토픽 정보
        topic_info = topic_model.get_topic_info()
        print(f"\n발견된 토픽 수: {len(topic_info) - 1}")  # -1은 outlier 제외
        print("\n토픽 정보:")
        print(topic_info.head(10))

        # 각 토픽의 대표 단어
        print(f"\n토픽별 대표 키워드:")
        for topic_id in topic_model.get_topics():
            if topic_id != -1:  # outlier 제외
                words = topic_model.get_topic(topic_id)
                top_words = [word for word, _ in words[:10]]
                print(f"  Topic {topic_id}: {', '.join(top_words)}")

        # 결과 저장
        bertopic_results[cluster_id] = {
            'model': topic_model,
            'topics': topics,
            'probs': probs,
            'topic_info': topic_info,
            'cluster_texts': cluster_texts,
            'cluster_df': cluster_df,
            'timestamp': datetime.now().strftime('%Y-%m-%d %H:%M:%S')
        }

        # 개별 Pickle 저장
        with open(f'step4_bertopic_cluster_{cluster_id}.pkl', 'wb') as f:
            pickle.dump(bertopic_results[cluster_id], f)

        # 토픽 시각화 저장
        try:
            fig = topic_model.visualize_topics()
            fig.write_html(f'bertopic_cluster_{cluster_id}_topics.html')
            print(f"✅ 시각화 저장: bertopic_cluster_{cluster_id}_topics.html")
        except:
            print("시각화 저장 실패 (건너뜀)")

        print(f"✅ Cluster {cluster_id} BERTopic 완료!")

    except Exception as e:
        print(f" Cluster {cluster_id} BERTopic 실패: {e}")
        continue

In [ ]:
#전체 결과 저장
# 전체 요약 정보만 저장 (모델 제외 - 용량 절약)
summary_results = {}
for cluster_id, result in bertopic_results.items():
    summary_results[cluster_id] = {
        'topic_info': result['topic_info'],
        'num_topics': len(result['topic_info']) - 1,
        'num_documents': len(result['cluster_texts']),
        'timestamp': result['timestamp']
    }

save_data_step4 = {
    'summary_results': summary_results,
    'optimal_k': optimal_k,
    'total_documents': len(df_clustered),
    'timestamp': datetime.now().strftime('%Y-%m-%d %H:%M:%S')
}

with open('step4_bertopic_summary.pkl', 'wb') as f:
    pickle.dump(save_data_step4, f)

print(f"Step 4 저장 완료:")
print(f"   - step4_bertopic_summary.pkl (전체 요약)")
for cluster_id in bertopic_results.keys():
    file_size = os.path.getsize(f'step4_bertopic_cluster_{cluster_id}.pkl') / (1024**2)
    print(f"   - step4_bertopic_cluster_{cluster_id}.pkl ({file_size:.1f} MB)")

In [ ]:
#최종 분석
for cluster_id, result in bertopic_results.items():
    topic_count = len(result['topic_info']) - 1
    doc_count = len(result['cluster_texts'])
    print(f"\nCluster {cluster_id}:")
    print(f"  - 문서 수: {doc_count:,}개")
    print(f"  - 토픽 수: {topic_count}개")

    # 상위 3개 토픽
    top_topics = result['topic_info'].head(4)  # 0은 outlier
    if len(top_topics) > 1:
        print(f"  - 주요 토픽:")
        for idx, row in top_topics.iterrows():
            if row['Topic'] != -1:
                print(f"    Topic {row['Topic']}: {row['Count']}개 문서")

print("Step 4 완료 (전체 파이프라인 종료)")

print(f"""
모든 단계 완료

생성된 파일:
Step 1: step1_preprocessing_result.pkl
Step 2: step2_embeddings_result.pkl
Step 3: step3_clustering_result.pkl
Step 4: step4_bertopic_cluster_*.pkl (클러스터별)
Step 4: step4_bertopic_summary.pkl (전체 요약)
CSV: clustering_result.csv
그래프: optimal_k_plot.png
시각화: bertopic_cluster_*_topics.html
""")